In [3]:
import numpy as np

In [7]:
from vllm import LLM, SamplingParams
import pandas as pd
import gc

ModuleNotFoundError: No module named 'vllm'

In [4]:
# Define your model path
model_path = "/home/data/Qwen/Qwen2.5-1.5B-Instruct"

In [3]:
# Create the LLM instance
llm = LLM(model=model_path, trust_remote_code=True)

INFO 08-01 06:25:27 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 08-01 06:25:28 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 08-01 06:25:29 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/home/bruceli/projects/data/Qwen/Qwen3-1.7B', speculative_config=None, tokenizer='/home/bruceli/projects/data/Qwen/Qwen3-1.7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_h

2025-08-01 06:25:30,894 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


WARNING 08-01 06:25:31 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7efe82181070>
INFO 08-01 06:25:31 [parallel_state.py:1004] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 08-01 06:25:31 [cuda.py:221] Using Flash Attention backend on V1 engine.
INFO 08-01 06:25:31 [topk_topp_sampler.py:44] Currently, FlashInfer top-p & top-k sampling sampler is disabled because FlashInfer>=v0.2.3 is not backward compatible. Falling back to the PyTorch-native implementation of top-p & top-k sampling.
INFO 08-01 06:25:31 [gpu_model_runner.py:1329] Starting to load model /home/bruceli/projects/data/Qwen/Qwen3-1.7B...


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  3.96it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  3.95it/s]



INFO 08-01 06:25:32 [loader.py:458] Loading weights took 0.58 seconds
INFO 08-01 06:25:32 [gpu_model_runner.py:1347] Model loading took 3.2152 GiB and 0.714961 seconds
INFO 08-01 06:25:39 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/bee35a0a39/rank_0_0 for vLLM's torch.compile
INFO 08-01 06:25:39 [backends.py:430] Dynamo bytecode transform time: 6.87 s
INFO 08-01 06:25:43 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 4.000 s
INFO 08-01 06:25:45 [monitor.py:33] torch.compile takes 6.87 s in total
INFO 08-01 06:25:45 [kv_cache_utils.py:634] GPU KV cache size: 579,040 tokens
INFO 08-01 06:25:45 [kv_cache_utils.py:637] Maximum concurrency for 40,960 tokens per request: 14.14x
INFO 08-01 06:26:03 [gpu_model_runner.py:1686] Graph capturing finished in 18 secs, took 0.47 GiB
INFO 08-01 06:26:03 [core.py:159] init engine (profile, create kv cache, warmup model) took 31.36 seconds
INFO 08-01 06:26:03 [core_client.py:4

In [6]:
# Sampling parameters
sampling_params = SamplingParams(
    n=1,
    temperature=0.0,  # temperature=0.0 means Deterministic
    top_p=1.0,
    max_tokens=50,
    logprobs=0,  # Request logprobs for each token
    prompt_logprobs=0,
)

NameError: name 'SamplingParams' is not defined

In [5]:
prompt1 = "What is the capital of France?"
# prompt2 = "What is the average size of a domestic cat?"

In [8]:
# Run generation
outputs = llm.generate([prompt1], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.25it/s, est. speed input: 22.91 toks/s, output: 163.60 toks/s]


In [9]:
# Print tokens and log probabilities
for output in outputs:
    print(f"Prompt: {output.prompt}")
    print("Generated Output:")
    # for token, logprob in zip(output.outputs[0].token_ids, output.outputs[0].logprobs):
    for logprob in output.outputs[0].logprobs:
        # decoded_token = llm.get_tokenizer().decode([token])
        print(f"{logprob}")

Prompt: What is the capital of France?
Generated Output:
{7281: Logprob(logprob=-1.2939517498016357, rank=1, decoded_token='ĠAlso')}
{11: Logprob(logprob=-7.867782187531702e-06, rank=1, decoded_token=',')}
{646: Logprob(logprob=-1.0789486169815063, rank=2, decoded_token='Ġcan')}
{498: Logprob(logprob=-0.000494715350214392, rank=1, decoded_token='Ġyou')}
{3291: Logprob(logprob=-0.06823074072599411, rank=1, decoded_token='Ġtell')}
{752: Logprob(logprob=-0.00011002412065863609, rank=1, decoded_token='Ġme')}
{911: Logprob(logprob=-0.11912998557090759, rank=1, decoded_token='Ġabout')}
{279: Logprob(logprob=-0.06143637374043465, rank=1, decoded_token='Ġthe')}
{11245: Logprob(logprob=-2.6074867248535156, rank=1, decoded_token='Ġfamous')}
{8585: Logprob(logprob=-1.4622935056686401, rank=1, decoded_token='ĠFrench')}
{29481: Logprob(logprob=-1.6184898614883423, rank=1, decoded_token='Ġpainter')}
{879: Logprob(logprob=-0.2603050172328949, rank=1, decoded_token='Ġwho')}
{23983: Logprob(logprob=-0.

In [70]:
type(outputs), len(outputs)  # len(outputs) == number of prompts

(list, 1)

In [71]:
outputs

[RequestOutput(request_id=4, prompt='What is the capital of France?', prompt_token_ids=[3838, 374, 279, 6722, 315, 9625, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=[None, {374: Logprob(logprob=-2.314427375793457, rank=2, decoded_token='Ġis')}, {279: Logprob(logprob=-0.1128106340765953, rank=1, decoded_token='Ġthe')}, {6722: Logprob(logprob=-8.558416366577148, rank=281, decoded_token='Ġcapital')}, {315: Logprob(logprob=-0.10791323333978653, rank=1, decoded_token='Ġof')}, {9625: Logprob(logprob=-3.009153127670288, rank=2, decoded_token='ĠFrance')}, {30: Logprob(logprob=-2.6586267948150635, rank=2, decoded_token='?')}], outputs=[CompletionOutput(index=0, text=' Also, can you tell me about the famous French painter who painted the Mona Lisa? What is the largest planet in our solar system? What is the chemical symbol for water? What is the capital of Italy? What is the largest country in the world', token_ids=[7281, 11, 646, 498, 3291, 752, 911, 279, 11245, 858

In [72]:
[m for m in dir(outputs[0]) if not m.startswith('__')]

['add',
 'encoder_prompt',
 'encoder_prompt_token_ids',
 'finished',
 'from_seq_group',
 'lora_request',
 'metrics',
 'multi_modal_placeholders',
 'num_cached_tokens',
 'outputs',
 'prompt',
 'prompt_logprobs',
 'prompt_token_ids',
 'request_id']

In [73]:
outputs[0].prompt_logprobs

[None,
 {374: Logprob(logprob=-2.314427375793457, rank=2, decoded_token='Ġis')},
 {279: Logprob(logprob=-0.1128106340765953, rank=1, decoded_token='Ġthe')},
 {6722: Logprob(logprob=-8.558416366577148, rank=281, decoded_token='Ġcapital')},
 {315: Logprob(logprob=-0.10791323333978653, rank=1, decoded_token='Ġof')},
 {9625: Logprob(logprob=-3.009153127670288, rank=2, decoded_token='ĠFrance')},
 {30: Logprob(logprob=-2.6586267948150635, rank=2, decoded_token='?')}]

In [74]:

for token_id in outputs[0].prompt_token_ids:
    print(f'{token_id:6d} {llm.get_tokenizer().decode([token_id])}')

  3838 What
   374  is
   279  the
  6722  capital
   315  of
  9625  France
    30 ?


In [75]:
outputs[0].outputs  # outputs[prompt_idx].outputs[response_idx]

[CompletionOutput(index=0, text=' Also, can you tell me about the famous French painter who painted the Mona Lisa? What is the largest planet in our solar system? What is the chemical symbol for water? What is the capital of Italy? What is the largest country in the world', token_ids=[7281, 11, 646, 498, 3291, 752, 911, 279, 11245, 8585, 29481, 879, 23983, 279, 98783, 28556, 30, 3555, 374, 279, 7772, 11580, 304, 1039, 12941, 1849, 30, 3555, 374, 279, 11483, 7735, 369, 3015, 30, 3555, 374, 279, 6722, 315, 15344, 30, 3555, 374, 279, 7772, 3146, 304, 279, 1879], cumulative_logprob=-21.122595513572037, logprobs=[{7281: Logprob(logprob=-1.2939517498016357, rank=1, decoded_token='ĠAlso')}, {11: Logprob(logprob=-7.867782187531702e-06, rank=1, decoded_token=',')}, {646: Logprob(logprob=-1.0789486169815063, rank=2, decoded_token='Ġcan')}, {498: Logprob(logprob=-0.000494715350214392, rank=1, decoded_token='Ġyou')}, {3291: Logprob(logprob=-0.06823074072599411, rank=1, decoded_token='Ġtell')}, {75

In [76]:
[m for m in dir(outputs[0].outputs[0]) if not m.startswith('__')]

['cumulative_logprob',
 'finish_reason',
 'finished',
 'index',
 'logprobs',
 'lora_request',
 'stop_reason',
 'text',
 'token_ids']

In [77]:
outputs[0].outputs[0].logprobs, len(outputs[0].outputs[0].logprobs)

([{7281: Logprob(logprob=-1.2939517498016357, rank=1, decoded_token='ĠAlso')},
  {11: Logprob(logprob=-7.867782187531702e-06, rank=1, decoded_token=',')},
  {646: Logprob(logprob=-1.0789486169815063, rank=2, decoded_token='Ġcan')},
  {498: Logprob(logprob=-0.000494715350214392, rank=1, decoded_token='Ġyou')},
  {3291: Logprob(logprob=-0.06823074072599411, rank=1, decoded_token='Ġtell')},
  {752: Logprob(logprob=-0.00011002412065863609, rank=1, decoded_token='Ġme')},
  {911: Logprob(logprob=-0.11912998557090759, rank=1, decoded_token='Ġabout')},
  {279: Logprob(logprob=-0.06143637374043465, rank=1, decoded_token='Ġthe')},
  {11245: Logprob(logprob=-2.6074867248535156, rank=1, decoded_token='Ġfamous')},
  {8585: Logprob(logprob=-1.4622935056686401, rank=1, decoded_token='ĠFrench')},
  {29481: Logprob(logprob=-1.6184898614883423, rank=1, decoded_token='Ġpainter')},
  {879: Logprob(logprob=-0.2603050172328949, rank=1, decoded_token='Ġwho')},
  {23983: Logprob(logprob=-0.3911629617214203, r

In [78]:
outputs[0].outputs[0].logprobs[10][29481].logprob

-1.6184898614883423

In [79]:
outputs[0].outputs[0].token_ids

[7281,
 11,
 646,
 498,
 3291,
 752,
 911,
 279,
 11245,
 8585,
 29481,
 879,
 23983,
 279,
 98783,
 28556,
 30,
 3555,
 374,
 279,
 7772,
 11580,
 304,
 1039,
 12941,
 1849,
 30,
 3555,
 374,
 279,
 11483,
 7735,
 369,
 3015,
 30,
 3555,
 374,
 279,
 6722,
 315,
 15344,
 30,
 3555,
 374,
 279,
 7772,
 3146,
 304,
 279,
 1879]

In [80]:


cum_logprob = 0
for d in outputs[0].outputs[0].logprobs:
    for k,v in d.items():
        cum_logprob += v.logprob

assert cum_logprob == outputs[0].outputs[0].cumulative_logprob


In [81]:
outputs[0].outputs[0].finished()

True

In [82]:
[m for m in dir(llm) if not m.startswith('_')]

['DEPRECATE_INIT_POSARGS',
 'DEPRECATE_LEGACY',
 'apply_model',
 'beam_search',
 'chat',
 'classify',
 'collective_rpc',
 'default_sampling_params',
 'deprecate_legacy_api',
 'embed',
 'encode',
 'engine_class',
 'generate',
 'get_default_sampling_params',
 'get_tokenizer',
 'llm_engine',
 'request_counter',
 'reset_prefix_cache',
 'score',
 'set_tokenizer',
 'sleep',
 'start_profile',
 'stop_profile',
 'wake_up']

In [83]:
llm.get_tokenizer()

CachedQwen2TokenizerFast(name_or_path='/home/bruceli/projects/data/Qwen/Qwen3-1.7B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='left', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=Fal

In [85]:
del llm
gc.collect()

1447

In [84]:
outputs_vllm = outputs.copy()

# SGLang

In [2]:
# launch the offline engine
import sglang as sgl

from sglang.srt.conversation import chat_templates
from sglang.test.test_utils import is_in_ci
from sglang.utils import async_stream_and_merge, stream_and_merge

if is_in_ci():
    import patch
else:
    import nest_asyncio

    nest_asyncio.apply()


import msgspec  # convert vLLM style sampling params to dict for SGLang

/root/anaconda3/envs/segment_wsl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm_sgl = sgl.Engine(model_path=model_path)
llm_sgl

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.29it/s]

Capturing batches (bs=1 avail_mem=7.24 GB): 100%|██████████| 23/23 [00:22<00:00,  1.02it/s] 


In [6]:
from transformers import AutoTokenizer

# 你的 model_path 前面已定义
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)


In [7]:
import json

def show_meta(ret, max_show=20):
    # llm_sgl.generate 单条返回是 dict；批量则是 list[dict]
    item = ret[0] if isinstance(ret, list) else ret
    meta = item.get("meta_info", {})
    print("=== meta_info keys ===")
    print(list(meta.keys()))

    print("\n=== finish_reason ===")
    print(json.dumps(meta.get("finish_reason", {}), indent=2))

    # 输出 token 级 logprob（前 max_show 个）
    otl = meta.get("output_token_logprobs", [])
    print(f"\n=== output_token_logprobs (前{max_show}条) ===")
    print(otl[:max_show])

    # 如果你把 logprob_start_len=0（见下一步），通常也会有 input_token_logprobs
    itl = meta.get("input_token_logprobs", [])
    print(f"\n=== input_token_logprobs (前{max_show}条) ===")
    print(itl[:max_show])

    # 还原输出文本看一下
    output_token_ids = [x[1] for x in otl]
    print("\n=== decoded output text ===")
    print(tokenizer.decode(output_token_ids, skip_special_tokens=False))


In [12]:
prompt = "Explain briefly what RLHF is."

# 推荐直接传 input_ids，避免模板差异
input_ids = tokenizer.encode(prompt, add_special_tokens=False)

ret = llm_sgl.generate(
    input_ids=input_ids,
    sampling_params={
        "temperature": 1.0,      # 你要非贪心就设 1.0
        "top_p": 1.0,
        "max_new_tokens": 64,
    },
    return_logprob=True,
    logprob_start_len=0,         # 重点：你的“start_len=0”
    # 可选：想拿 top-k 备选的 logprobs 就加下面这行
    # top_logprobs_num=5,
)

show_meta(ret)


=== meta_info keys ===
['id', 'finish_reason', 'prompt_tokens', 'input_token_logprobs', 'output_token_logprobs', 'completion_tokens', 'cached_tokens', 'e2e_latency']

=== finish_reason ===
{
  "type": "length",
  "length": 64
}

=== output_token_logprobs (前20条) ===
[(-3.0709030628204346, 81917, None), (-0.4223562479019165, 26753, None), (-7.707350254058838, 1380, None), (-1.1138958930969238, 432, None), (-2.1303491592407227, 646, None), (-0.024493133649230003, 387, None), (-0.5073946714401245, 1483, None), (-3.182177782058716, 304, None), (-4.240322113037109, 6236, None), (-0.6424704790115356, 61905, None), (-1.0359283685684204, 13, None), (-3.9328532218933105, 2585, None), (-5.238398551940918, 1657, None), (-8.262171745300293, 14056, None), (-5.827347755432129, 1052, None), (-0.1627182811498642, 525, None), (-0.24745145440101624, 304, None), (-6.8705034255981445, 422, None), (-8.06794548034668, 3767, None), (-13.662675857543945, 9555, None)]

=== input_token_logprobs (前20条) ===
[(None

In [8]:
# 1) 准备 prompt
prompt = "What is the capital of France?"
input_ids = tokenizer.encode(prompt, add_special_tokens=False)

# 2) 调用 llm_sgl.generate
result = llm_sgl.generate(
    input_ids=input_ids,
    sampling_params={
        "temperature": 1.0,
        "top_p": 1.0,
        "max_new_tokens": 32,
    },
    return_logprob=True,
    logprob_start_len=0,   # 关键参数
)

# 3) 提取 meta_info 和你关心的字段
meta_info = result["meta_info"]

output_tokens = [x[1] for x in meta_info["output_token_logprobs"]]
output_logprobs = [x[0] for x in meta_info["output_token_logprobs"]]

# 4) 打印出来
print("=== meta_info ===")
print(meta_info)

print("\n=== output_tokens ===")
print(output_tokens)

print("\n=== output_logprobs ===")
print(output_logprobs)

# 5) 如果要解码成文字看看效果：
decoded_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print("\n=== decoded text ===")
print(decoded_text)


=== meta_info ===
{'id': '844e3ef4a4db410eadb2c093db0cdacd', 'finish_reason': {'type': 'length', 'length': 32}, 'prompt_tokens': 7, 'input_token_logprobs': [(None, 3838, None), (-0.2663426697254181, 374, None), (-0.12340015918016434, 279, None), (-7.434597015380859, 6722, None), (-0.27008968591690063, 315, None), (-3.692899227142334, 9625, None), (-1.6525920629501343, 30, None)], 'output_token_logprobs': [(-0.7162180542945862, 576, None), (-0.014417066238820553, 6722, None), (-0.017033949494361877, 315, None), (-0.0002579425636213273, 9625, None), (-0.0032939265947788954, 374, None), (-0.0007614411297254264, 12095, None), (-2.1135082244873047, 382, None), (-9.087616920471191, 8078, None), (-6.802966594696045, 1493, None), (-2.899170398712158, 1378, None), (-0.2547663748264313, 22870, None), (-1.2431786060333252, 1119, None), (-0.6087610721588135, 264, None), (-2.2072386741638184, 3405, None), (-1.5188319683074951, 13, None), (-2.4839038848876953, 220, None), (-0.02767791412770748, 16, 

In [ ]:
# === 第一次生成的结果已经有了 ===
print("\n=== Step 1 decoded ===")
print(decoded_text)

# === 拼接输出到 input_ids 再生成一次 ===
input_ids2 = input_ids + output_tokens  # 把上一次生成的结果拼回去

result2 = llm_sgl.generate(
    input_ids=input_ids2,
    sampling_params={
        "temperature": 1.0,
        "top_p": 1.0,
        "max_new_tokens": 32,
    },
    return_logprob=True,
    logprob_start_len=0,
)

meta_info2 = result2["meta_info"]

output_tokens2 = [x[1] for x in meta_info2["output_token_logprobs"]]
output_logprobs2 = [x[0] for x in meta_info2["output_token_logprobs"]]

print("\n=== meta_info (Step 2) ===")
print(meta_info2)

print("\n=== output_tokens (Step 2) ===")
print(output_tokens2)

print("\n=== output_logprobs (Step 2) ===")
print(output_logprobs2)

decoded_text2 = tokenizer.decode(output_tokens2, skip_special_tokens=False)
print("\n=== Step 2 decoded ===")
print(decoded_text2)

print()



=== Step 1 decoded ===
 The capital of France is Paris.

Who was the Emperor of France during World War 1 and World War 2?  The Emperor of France during World War

=== meta_info (Step 2) ===
{'id': '9bfad6bf11d14bb48bf1ba8110d98815', 'finish_reason': {'type': 'length', 'length': 32}, 'prompt_tokens': 39, 'input_token_logprobs': [(None, 3838, None), (-0.2639544904232025, 374, None), (-0.1274380087852478, 279, None), (-7.43554162979126, 6722, None), (-0.3023774325847626, 315, None), (-3.718020439147949, 9625, None), (-1.646174669265747, 30, None), (-0.673613965511322, 576, None), (-0.017382539808750153, 6722, None), (-0.017033590003848076, 315, None), (-0.0002907091984525323, 9625, None), (-0.0032928551081568003, 374, None), (-0.0008861667010933161, 12095, None), (-2.2133519649505615, 382, None), (-5.094367027282715, 15191, None), (-1.653369665145874, 572, None), (-0.3175278902053833, 279, None), (-5.7954912185668945, 34314, None), (-0.19512706995010376, 315, None), (-0.6719577312469482

In [9]:
from pprint import pprint

In [18]:
pprint(meta_info2)

{'cached_tokens': 0,
 'completion_tokens': 32,
 'e2e_latency': 0.16212677955627441,
 'finish_reason': {'length': 32, 'type': 'length'},
 'id': '9bfad6bf11d14bb48bf1ba8110d98815',
 'input_token_logprobs': [(None, 3838, None),
                          (-0.2639544904232025, 374, None),
                          (-0.1274380087852478, 279, None),
                          (-7.43554162979126, 6722, None),
                          (-0.3023774325847626, 315, None),
                          (-3.718020439147949, 9625, None),
                          (-1.646174669265747, 30, None),
                          (-0.673613965511322, 576, None),
                          (-0.017382539808750153, 6722, None),
                          (-0.017033590003848076, 315, None),
                          (-0.0002907091984525323, 9625, None),
                          (-0.0032928551081568003, 374, None),
                          (-0.0008861667010933161, 12095, None),
                          (-2.213351964950

In [ ]:
input_tokens2 = [x[1] for x in meta_info2["input_token_logprobs"]]

input_logprobs2 = [x[0] for x in meta_info2["input_token_logprobs"]]

decoded_input_text2 = tokenizer.decode(input_tokens2, skip_special_tokens=False)

input_start_len_0 = dict(
    input_tokens = input_tokens2,
    input_logprobs = input_logprobs2,
    decoded_input_text = decoded_input_text2,
    output_tokens = [x[1] for x in meta_info2["output_token_logprobs"]]
    output_logprobs = [x[0] for x in meta_info2["output_token_logprobs"]],

)

pprint(input_start_len_0)

{'decoded_input_text': 'What is the capital of France? The capital of France '
                       'is Paris.\n'
                       '\n'
                       'Who was the Emperor of France during World War 1 and '
                       'World War 2?  The Emperor of France during World War',
 'input_logprobs': [None,
                    -0.2639544904232025,
                    -0.1274380087852478,
                    -7.43554162979126,
                    -0.3023774325847626,
                    -3.718020439147949,
                    -1.646174669265747,
                    -0.673613965511322,
                    -0.017382539808750153,
                    -0.017033590003848076,
                    -0.0002907091984525323,
                    -0.0032928551081568003,
                    -0.0008861667010933161,
                    -2.2133519649505615,
                    -5.094367027282715,
                    -1.653369665145874,
                    -0.3175278902053833,
          

In [62]:
result_start_len_at_prompt_end = llm_sgl.generate(
    input_ids=input_ids2,
    sampling_params={
        "temperature": 1.0,
        "top_p": 1.0,
        "max_new_tokens": 32,
    },
    return_logprob=True,
    logprob_start_len=0,
)

In [10]:
def generate_response(input_ids, temperature=1., start_len=0):

    response = llm_sgl.generate(
    input_ids=input_ids,
    sampling_params={
        "temperature": temperature,
        "top_p": 1.0,
        "max_new_tokens": 32,
    },
    return_logprob=True,
    logprob_start_len=start_len,
    )   


    meta_info = response["meta_info"]

    input_tokens = [x[1] for x in meta_info["input_token_logprobs"]]

    input_logprobs = [x[0] for x in meta_info["input_token_logprobs"]]

    decoded_input_text = tokenizer.decode(input_tokens, skip_special_tokens=False)

    result = dict(
        input_tokens = input_tokens,
        input_logprobs = input_logprobs,
        decoded_input_text = decoded_input_text,
        output_tokens = [x[1] for x in meta_info["output_token_logprobs"]],
        output_logprobs = [x[0] for x in meta_info["output_token_logprobs"]],
    )

    return result

In [ ]:
results = []

for i in range(2):
    results.append(generate_response(input_ids, temperature=0, start_len=3))

print(np.allclose(results[0]['output_logprobs'], results[1]['output_logprobs']))
print(np.allclose(results[0]['input_logprobs'][1:], results[1]['input_logprobs'][1:]))

'''
Conclusion:
same parameter run twice (no matter what temperature is):
- same input logprob at the same token
'''

'''
Conclusion:
same parameter run twice (temperature=0):
- same output logprob at the same token
'''


False
True


'\nConclusion:\nsame parameter run twice:\n- same input logprob at the same token\n'

In [ ]:
res1 = generate_response(input_ids, temperature=0, start_len=3)
res2 = generate_response(input_ids, temperature=0, start_len=0)
print(res1['decoded_input_text'], res2['decoded_input_text'])
print(res1['output_logprobs'][-5:], res2['output_logprobs'][-5:])
print(res1['input_logprobs'], res2['input_logprobs'])

'''
Conclusion:
same input, same temperature (even at 0), different start length leads to:
- different input logprob at the same token
- different output logprob at the same token
'''


 capital of France? What is the capital of France?
[-0.038739826530218124, -0.006982094142585993, -1.2796893119812012, -0.06121796742081642, -0.16203784942626953] [-0.03871792182326317, -0.006087571382522583, -1.3280937671661377, -0.05748198926448822, -0.15814797580242157]
[None, -0.30019545555114746, -3.701012134552002, -1.647252082824707] [None, -0.2663426697254181, -0.12340015918016434, -7.434597015380859, -0.27008968591690063, -3.692899227142334, -1.6525920629501343]


In [15]:
res1 = generate_response(input_ids, temperature=0, start_len=3)
res2 = generate_response(input_ids + tokenizer.encode('My name is Elizabeth.'), temperature=0, start_len=3)
print(res1['decoded_input_text'], res2['decoded_input_text'])
print(res1['output_logprobs'][-5:], res2['output_logprobs'][-5:])




 capital of France?  capital of France?My name is Elizabeth.
[-0.038739826530218124, -0.006982094142585993, -1.2796893119812012, -0.06121796742081642, -0.16203784942626953] [-0.556067705154419, -2.174212694168091, -0.519115686416626, -0.9894031286239624, -0.25920653343200684]


In [18]:
[res1['input_logprobs'], res2['input_logprobs']]

[[None, -0.30019545555114746, -3.701012134552002, -1.647252082824707],
 [None,
  -0.3002954125404358,
  -3.7182364463806152,
  -1.6308228969573975,
  -14.545310020446777,
  -3.7796692848205566,
  -0.03700103610754013,
  -7.269960880279541,
  -1.4145187139511108]]

In [ ]:
tokenizer.encode(' capital of France?')

[6722, 315, 9625, 30]

In [85]:
tokenizer.encode('What is the capital of France?')

[3838, 374, 279, 6722, 315, 9625, 30]

In [38]:
input_logprobs3[-5:], input_logprobs2[-5:]

([-0.00789888110011816,
  -0.0006735440110787749,
  -0.04346011206507683,
  -0.01717608980834484,
  -0.0045041777193546295],
 [-0.007927857339382172,
  -0.0006526962388306856,
  -0.04844208061695099,
  -0.019290227442979813,
  -0.005093692801892757])

In [48]:
input_start_len_at_prompt_end_run1 = input_start_len_at_prompt_end.copy()

In [49]:
input_start_len_at_prompt_end_run1

{'input_tokens': [3838,
  374,
  279,
  6722,
  315,
  9625,
  30,
  576,
  6722,
  315,
  9625,
  374,
  12095,
  382,
  15191,
  572,
  279,
  34314,
  315,
  9625,
  2337,
  4337,
  5004,
  220,
  16,
  323,
  4337,
  5004,
  220,
  17,
  30,
  220,
  576,
  34314,
  315,
  9625,
  2337,
  4337,
  5004],
 'input_logprobs': [None,
  -0.2639544904232025,
  -0.1274380087852478,
  -7.43554162979126,
  -0.3023774325847626,
  -3.718020439147949,
  -1.646174669265747,
  -0.673613965511322,
  -0.017382539808750153,
  -0.017033590003848076,
  -0.0002907091984525323,
  -0.0032928551081568003,
  -0.0008861667010933161,
  -2.2133519649505615,
  -5.094367027282715,
  -1.653369665145874,
  -0.3175278902053833,
  -5.7954912185668945,
  -0.19512706995010376,
  -0.6719577312469482,
  -1.0889030694961548,
  -1.7123857736587524,
  -0.0009180859779007733,
  -2.5197296142578125,
  -0.49349820613861084,
  -3.319289207458496,
  -1.2153327465057373,
  -0.02170300856232643,
  -0.04399745166301727,
  -0.0028

In [52]:
input_start_len_at_prompt_end

{'input_tokens': [3838,
  374,
  279,
  6722,
  315,
  9625,
  30,
  576,
  6722,
  315,
  9625,
  374,
  12095,
  382,
  15191,
  572,
  279,
  34314,
  315,
  9625,
  2337,
  4337,
  5004,
  220,
  16,
  323,
  4337,
  5004,
  220,
  17,
  30,
  220,
  576,
  34314,
  315,
  9625,
  2337,
  4337,
  5004],
 'input_logprobs': [None,
  -0.2639544904232025,
  -0.1274380087852478,
  -7.43554162979126,
  -0.3023774325847626,
  -3.718020439147949,
  -1.646174669265747,
  -0.673613965511322,
  -0.017382539808750153,
  -0.017033590003848076,
  -0.0002907091984525323,
  -0.0032928551081568003,
  -0.0008861667010933161,
  -2.2133519649505615,
  -5.094367027282715,
  -1.653369665145874,
  -0.3175278902053833,
  -5.7954912185668945,
  -0.19512706995010376,
  -0.6719577312469482,
  -1.0889030694961548,
  -1.7123857736587524,
  -0.0009180859779007733,
  -2.5197296142578125,
  -0.49349820613861084,
  -3.319289207458496,
  -1.2153327465057373,
  -0.02170300856232643,
  -0.04399745166301727,
  -0.0028

In [56]:
input_start_len_at_prompt_end == input_start_len_at_prompt_end_run1

True

In [64]:
len(input_start_len_at_prompt_end['input_logprobs']), len(input_start_len_at_prompt_end_run1['input_logprobs']), 

(39, 39)

In [65]:
input_start_len_at_prompt_end['input_logprobs'][-5:], input_start_len_at_prompt_end_run1['input_logprobs'][-5:]

([-0.007927857339382172,
  -0.0006526962388306856,
  -0.04844208061695099,
  -0.019290227442979813,
  -0.005093692801892757],
 [-0.007927857339382172,
  -0.0006526962388306856,
  -0.04844208061695099,
  -0.019290227442979813,
  -0.005093692801892757])

In [66]:
input_start_len_at_prompt_end['output_logprobs'][-5:], input_start_len_at_prompt_end_run1['output_logprobs'][-5:]

KeyError: 'output_logprobs'